In [11]:
import numpy as np
import time

# Setting up parameters

## General parameters

In [4]:
pulsedmasterlogic.set_generation_parameters(
    laser_channel="d_ch0",
    laser_length=400e-9,
    gate_channel="d_ch1",
    laser_delay=0,
    microwave_channel="d_ch2",
    microwave_frequency=2.87e9,  
    microwave_amplitude=0.25,
    rabi_period=200e-9, 
    wait_time=1e-6,
    sync_channel="",
)

# Read-only access to all available and currently set global generation parameters as dictionary via property
for param, value in pulsedmasterlogic.generation_parameters.items():
    print('{0}: {1}'.format(param, value))

laser_channel: d_ch0
sync_channel: 
gate_channel: d_ch1
microwave_channel: d_ch2
microwave_frequency: 2870000000.0
microwave_amplitude: 0.25
rabi_period: 2e-07
laser_length: 5e-07
laser_delay: 0
wait_time: 1e-06
analog_trigger_voltage: 0.0


## Additional parameters

In [9]:
# Get the dictionary containing the default parameters for the desired sequence and alter the values as needed.
generate_params = pulsedmasterlogic.generate_method_params['rabi']
generate_params['name'] = 'my_rabi'
generate_params['tau_start'] = 50e-9
generate_params['tau_step'] = 100e-9
generate_params['num_of_points'] = 10

# Sample pulse block and upload to AWG

In [13]:
# The "with_load" flag can be used to also load the waveform into the channels after creating it
pulsedmasterlogic.sample_ensemble('my_rabi', with_load=True)
# Wait for the sampling to finish. There is a status dictionary that can be used to check for process status.
while pulsedmasterlogic.status_dict['sampload_busy']:
    time.sleep(0.1)

In [9]:
# You can check on the currently loaded asset on the pulse generator by using the following property:
print('currently loaded asset:', pulsedmasterlogic.loaded_asset)
# You can check on all created waveforms on the pulse generator by using the following property:
print('Available waveforms on device:', pulsedmasterlogic.sampled_waveforms)

currently loaded asset: ('', '')
Available waveforms on device: ['cwODMR_a_ch0', 'cwODMR_a_ch1', 'cwODMR_d_ch0', 'cwODMR_d_ch1', 'cwODMR_d_ch2', 'cwODMR_d_ch3', 'HILO50_a_ch0', 'HILO50_a_ch1', 'HILO50_a_ch2', 'HILO50_a_ch3', 'HILO50_d_ch0', 'HILO50_d_ch1', 'HILO50_d_ch2', 'HILO50_d_ch3', 'HILO50_d_ch4', 'HILO50_d_ch5', 'HILOBLOCK_a_ch0', 'HILOBLOCK_a_ch1', 'HILOBLOCK_a_ch2', 'HILOBLOCK_a_ch3', 'HILOBLOCK_d_ch0', 'HILOBLOCK_d_ch1', 'HILOBLOCK_d_ch2', 'HILOBLOCK_d_ch3', 'HILOBLOCK_d_ch4', 'HILOBLOCK_d_ch5', 'HILO_a_ch0', 'HILO_a_ch1', 'HILO_a_ch2', 'HILO_a_ch3', 'HILO_d_ch0', 'HILO_d_ch1', 'HILO_d_ch2', 'HILO_d_ch3', 'HILO_d_ch4', 'HILO_d_ch5', 'laser_empty_a_ch0', 'laser_empty_a_ch1', 'laser_empty_a_ch2', 'laser_empty_a_ch3', 'laser_empty_d_ch0', 'laser_empty_d_ch1', 'laser_empty_d_ch2', 'laser_empty_d_ch3', 'laser_empty_d_ch4', 'laser_empty_d_ch5', 'MW_trig_a_ch0', 'MW_trig_a_ch1', 'MW_trig_a_ch2', 'MW_trig_a_ch3', 'MW_trig_d_ch0', 'MW_trig_d_ch1', 'MW_trig_d_ch2', 'MW_trig_d_ch3', 'MW

# Set up measurement

In [ ]:
# If the pulse sequence has been generated not using predefined methods (e.g. using the graphical table editor) the 
# measurement settings have to be given manually.
# In the same way as setting the generation parameters you can either pass a settings dictionary or single keyword 
# arguments or a combination of both:
pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable=1e-8 + np.arange(50) * 1e-8,
                                           number_of_lasers=50, 
                                           laser_ignore_list=[], 
                                           alternating=False, 
                                           units=('s', 'arb. u.'))
time.sleep(0.5)

# If the pulse sequence to run has been generated using predefined methods, the measurement settings can be invoked 
# from the PulseBlockEnsemble attribute "measurement_information". In that case it is enough to execute the following:
pulsedmasterlogic.set_measurement_settings(invoke_settings=True)
time.sleep(0.5)

In [ ]:
# You can get a read-only view of the current measurement settings by using this property:
for setting, value in pulsedmasterlogic.measurement_settings.items():
    print('{0}:\n  {1}'.format(setting, value))

### Set up fast counter

In [ ]:
# In case you used "invoke_Settings" this part is not necessary
# The number_of_gates is only used in case of a hardware gated fast counter. It will always be set to 0 otherwise.
# The record length should be the length of the pulse sequence for an ungated counter and the length of the longest 
# gate for a gated counter.
pulsedmasterlogic.set_fast_counter_settings(record_length=237.8e-6, number_of_gates=50)
time.sleep(0.5)

In [ ]:
# You can see the current fast counter settings with the this property:
for setting, value in pulsedmasterlogic.fast_counter_settings.items():
    print('{0}: {1}'.format(setting, value))

### Set up extraction settings

In [ ]:
# You can also change how qudi will extract laser pulses from the raw fast counter timetrace.
# You can get all available extraction methods with the following property:
print(list(pulsedmasterlogic.extraction_methods))

In [ ]:
# To change the current extraction method:
pulsedmasterlogic.set_extraction_settings(method='threshold')
time.sleep(0.2)

# You can get the current extraction method along with all needed settings with the read-only property:
for setting, value in pulsedmasterlogic.extraction_settings.items():
    print('{0}: {1}'.format(setting, value))

In [ ]:
# Set the extraction settings in the same way as you set other settings
pulsedmasterlogic.set_extraction_settings(count_threshold=20, min_laser_length=100e-9, threshold_tolerance=10e-9)
time.sleep(0.2)

### Set up analysis settings

In [ ]:
# Completely analogous to the extraction method
print(list(pulsedmasterlogic.analysis_methods))

In [ ]:
# To change the current analysis method:
pulsedmasterlogic.set_analysis_settings(method='mean_norm')
time.sleep(0.2)

# You can get the current analysis method along with all needed settings with the read-only property:
for setting, value in pulsedmasterlogic.analysis_settings.items():
    print('{0}: {1}'.format(setting, value))

In [ ]:
# Set the analysis settings in the same way as you set other settings
pulsedmasterlogic.set_analysis_settings(signal_start=0.0, signal_end=2e-7, norm_start=4e-7, norm_end=7e-7)
time.sleep(0.2)

### Set up additional MW CW

In [ ]:
# Set new values
pulsedmasterlogic.set_ext_microwave_settings(power=-20, frequency=2.5e9, use_ext_microwave=True)
time.sleep(0.5)

# get the current settings with the following property:
for setting, value in pulsedmasterlogic.ext_microwave_settings.items():
    print('{0}: {1}'.format(setting, value))

### FFT plot

In [ ]:
pulsedmasterlogic.set_alternative_data_type('FFT')
time.sleep(0.5)
print(pulsedmasterlogic.alternative_data_type)

### Refresh rate

In [ ]:
# This will set the time interval for reading the raw data from the fast counter and analyzing it. 
# Don't set this interval too small since it can lead to heavy CPU load and crashes.
# If you want to update the data on demand you can always click the "pull and analyze data" button in the GUI.
pulsedmasterlogic.set_timer_interval(5)
time.sleep(0.2)
print(pulsedmasterlogic.timer_interval)

## Start pulsed meassurement

In [ ]:
# If everything is properly set, we can start a measurement simply by calling:
pulsedmasterlogic.toggle_pulsed_measurement(True)
# Wait until the pulsedmeasurementlogic is actually busy and the measurement is running
while pulsedmeasurementlogic.module_state() != 'locked':
    time.sleep(0.2)

## Stop pulsed measurement

In [ ]:
pulsedmasterlogic.toggle_pulsed_measurement(False)
# Wait until the pulsedmeasurementlogic is actually idle and the measurement is stopped
while pulsedmeasurementlogic.module_state() == 'locked':
    time.sleep(0.2)

## Save data

In [ ]:
pulsedmasterlogic.save_measurement_data(tag='my_dummy_measurement', with_error=True)